# Rendering images with different light source densities

In [ ]:
import jax
from jax import numpy as jnp

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) 

import os
import pickle
import sys

sys.path.append('../')

from rtnf.helpers import makedir, save_to_dir_np
from rtnf.truefield import get_X, interp_grid, plot_3d, generate_emission, init_rays, plot_2d
from rtnf.render import renderfn

os.environ['CUDA_VISIBLE_DEVICES'] = '3' # Restrict to 1 GPU

In [ ]:
# resolution for generating IoR Field
res_eta = 16
# resolution for grid of light sources
res_lum = 64 
# resolution (number of rays) for ray tracing
res = 64
# number of sources in light source field
num_sources = 50
# angle of sensor
fov_angle = 0.1

eta_shape = (res_eta, res_eta, res_eta) 
lum_shape = (res_lum, res_lum, res_lum)
res_shape = (res, res)

X_eta = get_X(res_eta)
X_lum = get_X(res_lum)

# Seed for IoR Field
seed = 9
rand_key = jax.random.PRNGKey(seed)

# Seed for Light Sources
seed_ls = 0

save = True #Whether or not to save output
save_dir = '../exp/src_density_%d/'%(num_sources) #Directory for saving truefield output

### Load IoR Field

In [ ]:
# Load IoR Field

grid_out = pickle.load(open('../data_paper/Multi_Gauss/s9/true_eta.p', 'rb'))
predict_eta = interp_grid(grid_out, cval=1.)

In [ ]:
%matplotlib widget

# visualize

plot_3d(grid_out, res_eta)

### Generate a light source field

In [ ]:
# Generate Source Locations
lum_field, predict_lum_grid, ls = generate_emission('uniform', seed_ls, X_lum, lum_shape, num_sources=num_sources)

### Visualize Light Sources

In [ ]:
%matplotlib widget

# Visualize
plot_3d(lum_field, res_lum)

### Generate Ray Initial Conditions from Sensor

In [ ]:
# Initial Conditions
rays = init_rays(fov_angle, res)
s_vals = jnp.linspace(0, 1.2, 3)
# Render the image
ray_trace, ray_lum = renderfn(rays, predict_eta, predict_lum_grid, s_vals)

### Render a Refracted Image

In [ ]:
%matplotlib inline

# Plot the image
plot_2d(ray_lum.sum(1).reshape(res, res)[::-1, ::-1].T, title='Luminance Image (Refracted)')

### Save Output for Training

In [ ]:
if save: 
    makedir(save_dir)
    save_to_dir_np(save_dir, X_eta, rays, s_vals, lum_field, ray_lum, grid_out)